In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# An Introduction to Static Typing in TypeScript

TypeScript is a superset of JavaScript that adds a robust static type system to the language. Code written in TypeScript is compiled (transpiled) into JavaScript. During this process, the TypeScript compiler analyzes the code to ensure type safety.

This allows developers to catch errors at compile-time, long before the code runs in a browser or on a server. The type system is designed to be flexible and expressive, supporting:

* **Primitive types** (number, string, boolean)
* **Generic types** for reusable components
* **Union types** for values that can take multiple forms
* **Recursive types** for complex data structures

By utilizing type annotations, code becomes self-documenting, easier to refactor, and safer to execute.

## Finding Errors via Static Analysis

In a TypeScript environment, type checking happens before execution.

Consider the following example. We have an input string "5".
If we tell TypeScript that we expect a **numerical result** from our calculation, it will protect us from accidental string concatenation.

In [ ]:
const inputString: string = "5";

const result: number = inputString + 1;
result;

We can fix this by parsing `inputString` to the type `number`:

In [ ]:
const inputString: string = "5";

const result: number = parseInt(inputString, 10) + 1;
result;

## Type Annotations

The core of TypeScript is specifying types for variables, function parameters, and return values. 
* **Parameters:** Annotated by placing a colon `:` followed by the type after the parameter name.
* **Return Type:** Specified after the parameter list parentheses.

In [ ]:
function add(a: number, b: number): number {
    return a + b;
}

If we attempt to call this function with incompatible types (e.g., strings), the TypeScript compiler will raise an error. This prevents invalid data from flowing through the application.

In [ ]:
const myName = "Karl";

add("Hello ", myName); 

In [ ]:
add(10,5);

## Type Erasure

A key concept in TypeScript is **Type Erasure**. Types exist only during development and compilation. Once the code is compiled to JavaScript, all type annotations, interfaces, and custom type definitions are removed. The runtime environment (Node.js, Browser) sees only pure JavaScript.

## Built-in Types

TypeScript supports all standard JavaScript primitives (`number`, `string`, `boolean`) and complex structures like Arrays (`number[]` or `Array<number>`).

The function `average` below demonstrates working with arrays of numbers.

In [ ]:
function average(numbers: number[]): number {
    if (numbers.length === 0) return 0;
    const total = numbers.reduce((acc, curr) => acc + curr, 0);
    return total / numbers.length;
}

In [ ]:
average([1, 2, 3, 4])

## Custom Types and Classes

We can define custom data structures using `class` or `interface`. In a TypeScript class, properties must be declared and initialized. The `public` keyword can be used to automatically define and assign properties in the constructor.

In [ ]:
class Person {
    name: string;

    constructor(name: string) {
        this.name = name;
    }

    greet(): string {
        return `Hello, ${this.name}!`;
    }
}

When a function performs an action but does not return a value, its return type is `void`.

In [ ]:
function salve(p: Person): void {
    console.log(p.greet());
}

In [ ]:
const jc = new Person('Julius Caesar');
salve(jc);

## Union Types and Type Narrowing

Sometimes a value can be of more than one type. The **Union Type** operator `|` allows us to define a variable that can hold, for example, either a `string` OR an object.

To work with these values safely, we use **Type Narrowing** (e.g., checking `typeof`) to determine the specific type at runtime.

In [ ]:
type NameDict = { given: string; family: string };

function greetName(name: string | NameDict): string {
    if (typeof name === 'string') {
        return 'Hi ' + name + '!';
    } 
    
    return `Bienvenido, Señor ${name.given} ${name.family}.`;
}

In [ ]:
greetName("Alice")

In [ ]:
greetName({ given: 'Esteban', family: 'Ramirez' })

## Escape Hatches

Sometimes, TypeScript's strictness can get in the way, or we might differ with the compiler about the type of a variable. TypeScript provides specific tools to handle uncertainties or to override the compiler's inference.

### 1. The `any` type (The "Off Switch")
The `any` type essentially disables type checking for a variable. It allows you to access any property or call any method. While useful for quick prototyping or migrating JavaScript code, it defeats the purpose of TypeScript and should be avoided when possible.

### 2. The `unknown` type (The Safe Alternative)
`unknown` is the type-safe counterpart to `any`. It represents a value that could be anything, but TypeScript won't let you perform arbitrary operations on it until you restrict the type (e.g., via `typeof` checks).

In [ ]:
let looseValue: any = 4;
looseValue = "hello";
looseValue.toUpperCase();

In [ ]:
let safeValue: unknown = "hello";
safeValue.toUpperCase();

In [ ]:
if (typeof safeValue === 'string') {
    console.log(safeValue.toUpperCase());
}

## Overriding the Compiler: `as` and `!`

While strict typing is safer, there are situations where we need to override the compiler's decisions. TypeScript provides mechanisms for this, but they should be used with **extreme caution**.

Using these operators effectively **turns off the type checker** for that specific expression. By using them, you take full responsibility for type safety. If you are wrong, the compiler will not warn you, and your program may crash at runtime.

### 1. Type Assertions (`as`)

The `as` keyword tells the compiler to treat a value as a specific type, regardless of what the compiler inferred.

* **When to use:** Only when you have external knowledge about a type that TypeScript cannot know (e.g., data coming from a specific API endpoint).
* **The Risk:** You are essentially "lying" to the compiler. If the actual value does not match your assertion, strict typing provides zero protection.

In [ ]:
let someValue: unknown = "hello world";

let numberLength = (someValue as number).toFixed(2);

### 2. The Non-Null Assertion Operator (`!`)

The exclamation mark `!` after a variable removes `null` and `undefined` from its type.

* **When to use:** When you are absolutely certain a value exists, even though the type definition says it might be missing.
* **The Risk:** If the value *is* actually `null` or `undefined` at runtime, accessing properties on it will cause the program to crash immediately.

In [ ]:
function findUser(id: number): string | undefined {
    return undefined;
}

const user = findUser(42)!;
user.toUpperCase();

## Generics

Generics allow us to write reusable code that works with a variety of types while maintaining strict type safety. We use type variables (conventionally `<T>`, `<S>`, etc.) to represent these future types.

The `swap` function below works with a tuple of two elements of potentially different types and swaps them.

In [ ]:
function swap<S, T>(pair: [S, T]): [T, S] {
    const [x, y] = pair;
    return [y, x];
}

TypeScript infers the generic types automatically based on the arguments passed. Here, `S` and `T` become `number`.

In [ ]:
swap([1, 2]);

Here, types are mixed. `S` is `number` and `T` is `string`. The return type is correctly inferred as `[string, number]`.

In [ ]:
swap([1, 'a']);

## Explicit Generics

We can define multiple generic type variables for more complex structures.

In [ ]:
function rotate<U, V, W>(triple: [U, V, W]): [V, W, U] {
    const [x, y, z] = triple;
    return [y, z, x];
}

rotate([1, 'a', true]);

## Recursive Types & Discriminated Unions

TypeScript excels at modeling recursive data structures, such as mathematical expressions.

We define an `Operator` type using string literals, and an `Expression` type that can be a number, a variable name (string), or a tuple containing nested expressions.

In [ ]:
type Operator = '+' | '-' | '*' | '/';

type Expression = number | string | [Expression, Operator, Expression];

The `differentiate` function computes the derivative of an expression. Since TypeScript types disappear at runtime, we cannot pattern match on the type itself. Instead, we inspect the runtime structure (checking if it's a number, string, or array) to guide the logic.

In [ ]:
function differentiate(expr: Expression, vari: string = 'x'): Expression {
    // Case 1: Constant (number) -> Derivative is 0
    if (typeof expr === 'number') {
        return 0.0;
    }

    // Case 2: Variable (string) -> Derivative is 1 if it matches 'var', else 0
    if (typeof expr === 'string') {
        return expr === vari ? 1.0 : 0.0;
    }

    // Case 3: Compound Expression (Tuple) -> Apply differentiation rules
    if (Array.isArray(expr)) {
        const [lhs, op, rhs] = expr;
        const dLhs = differentiate(lhs, vari);
        const dRhs = differentiate(rhs, vari);

        switch (op) {
            case '+':
                return [dLhs, '+', dRhs];
            case '-':
                return [dLhs, '-', dRhs];
            case '*':
                // Product Rule: (u*v)' = u'v + uv'
                const p1: Expression = [dLhs, '*', rhs];
                const p2: Expression = [lhs, '*', dRhs];
                return [p1, '+', p2];
            case '/':
                // Quotient Rule
                const q1: Expression = [dLhs, '*', rhs];
                const q2: Expression = [lhs, '*', dRhs];
                const numerator: Expression = [q1, '-', q2];
                const denominator: Expression = [rhs, '*', rhs];
                return [numerator, '/', denominator];
        }
    }

    return 0.0;
}

We can now test the differentiation logic on the expression $x^2 + 3x$ (represented as `x * x + 3 * x`). 
The expected result is equivalent to $2x + 3$.

In [ ]:
const myExpr: Expression = [['x', '*', 'x'], '+', ['3', '*', 'x']];
const resultExpr = differentiate(myExpr);

resultExpr;

## Advanced: Nominal Typing & Branded Types

By default, TypeScript uses **Structural Typing**. This means if two types have the same structure, they are compatible.
However, sometimes we want **Nominal Typing**—where the *name* or identity of the type matters, not just its structure.

For example, in a Finite Automaton, we might use a string as a key for transitions. We want to ensure that we don't accidentally pass a generic string (like a user's name) into a function that expects a specific `TransitionKey`.

To achieve this, we use a technique called **Branding**.

### The "Unique Symbol" Trick

We can create a "brand" using a `unique symbol`.
* **`symbol`**: A primitive type in JavaScript/TypeScript that is guaranteed to be unique.
* **`unique symbol`**: A TypeScript type that refers to a specific, unique symbol identity.
* **`declare`**: Tells TypeScript that a variable exists, but we don't need to define its runtime value. Here, we use it to create a "phantom property" that exists only in the type system, adding no overhead to the compiled JavaScript.

In [ ]:
declare const TransitionKeyBrand: unique symbol;

type TransitionKey = string & { [TransitionKeyBrand]: true };

type Value = number; 
type Char = string;

function key(q: Value, c: Char): TransitionKey {
    return `${q},${c}` as TransitionKey;
}

### Why do we do this?

Now, strict type safety prevents us from mixing up keys with normal strings.

* We **cannot** simply assign a string to a `TransitionKey`.
* We **must** use the `key()` function to create one.

In [ ]:
function processTransition(k: TransitionKey) {
    console.log("Processing transition:", k);
}

In [ ]:
const randomString = "0,a";

processTransition(randomString);

In [ ]:
const myKey = key(0, 'a');
processTransition(myKey);